<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Fitabase_Regression_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install glob2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from glob import glob
import os
import pandas as pd
path = "/content/drive/MyDrive/Iqram Sir/Fitabase data pulls - seperate"

In [30]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm

def get_content(patient_folder_adrs):

  post_adrs = sorted(glob(patient_folder_adrs+"/*"))[0]
  

  pre_adrs = sorted(glob(patient_folder_adrs+"/*"))[1]

  for i in glob(post_adrs+"/*"):
    content = i.split("_")
    if "1min" in content:
      post_1min = i
    if "minuteCaloriesNarrow" in content:
      post_cal = i
    if "minuteMETsNarrow" in content:
      post_met = i


  for i in glob(pre_adrs+"/*"):
    content = i.split("_")
    if "1min" in i.split("_"):
      pre_1min = i
    if "minuteCaloriesNarrow" in content:
      pre_cal = i
    if "minuteMETsNarrow" in content:
      pre_met = i
 
  return post_1min, post_cal,post_met, pre_1min, pre_cal, pre_met




def plot(X,Y,m,c,hr_aee = False):
  fig = plt.figure(figsize=(13,8))
  ax = fig.add_subplot()
  fig.subplots_adjust(top=0.85)

  # Set titles for the figure and the subplot respectively
  #fig.suptitle('bold figure suptitle', fontsize=14, fontweight='bold')
  if hr_aee == True:
    ax.set_title('Max HR & aEE',fontweight='bold')
  else:
    ax.set_title('Max HR',fontweight='bold')

  ax.scatter(X, Y)

  high_y = Y.max()

  high_x = (high_y-c)/m

  rang_y=[Y.min(),high_y]

  ax.plot((rang_y-c)/m, rang_y, color = "red")

  ax.scatter(high_x, high_y, color='green')
  ax.annotate('(aEEMax,HRMax)', xy=(high_x, high_y), xytext=(high_x-5, high_y+10),fontweight='bold')
  ax.annotate("("+str(round(high_x[0],2))+","+str(high_y)+")", xy=(high_x, high_y), xytext=(high_x-3, high_y+5),fontweight='bold',color = 'purple')


  if hr_aee == True:
    ax.set_xlabel("aEE_MAX(kJ/min)",fontweight='bold',fontsize=12)
    ax.set_ylabel("HR_MAX(BPM)",fontweight='bold',fontsize=12)
  else:
    ax.set_xlabel("aEE(kJ/min)",fontweight='bold',fontsize=12)
    ax.set_ylabel("HR(BPM)",fontweight='bold',fontsize=12)

  ax.axis([0, high_x+10, 0, high_y+30])

  ax.grid()
  plt.show()




def engine(df_1min, df_cal,df_met,min,hr_aee = False):
  list_1 = list(df_1min["Time"])
  list_2 = list(df_cal["ActivityMinute"])
  df_met = mets_minute_refine(df_met,min)
  list_3 = list(df_met["ActivityMinute"])


  hr = []
  aee = []
  time = []
  mets = []

  if len(list_1)>len(list_2):
    for i in tqdm(list_1):
      if i in list_2 and i in list_3:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        aee.append(list(df_cal.query("ActivityMinute == '"+i+"'")['Calories'])[0]*4.184)
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  else:
    for i in tqdm(list_2):
      if i in list_1 and i in list_3:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        aee.append(list(df_cal.query("ActivityMinute == '"+i+"'")['Calories'])[0]*4.184)
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  d={
    "Time" : time,
     "HR(BPM)" : hr,
     "aEE(kJ/min)":aee,
     "METs" : mets
  }

  dataFrame = pd.DataFrame.from_dict(d)


  Y = dataFrame["HR(BPM)"].values.reshape(-1, 1)
  X = dataFrame["aEE(kJ/min)"].values.reshape(-1, 1)
  reg = LinearRegression().fit(X, Y)
  m = reg.coef_[0]
  c = reg.intercept_

  if hr_aee == True:
    hr_mx = Y.max()
    aee_mx = (hr_mx-c)/m
    return hr_mx, round(aee_mx[0],2)

  else:
    plot(X,Y,m,c)
    return dataFrame



def test_signle_sample(data,df_1min,df_cal,df_met):
  total_row = data.shape[0]
  print("Total DataRow: ",total_row)

  err_row=[]

  for i in tqdm(range(total_row)):
    print(data["Time"][i], data['HR(BPM)'][i],round(data['aEE(kJ/min)'][i],2),data['METs'][i])

    hr = list(df_1min.query("Time == '"+data["Time"][i]+"'")['Value'])[0]
    cal = round(list(df_cal.query("ActivityMinute == '"+data["Time"][i]+"'")['Calories'])[0],2)
    mets = list(df_met.query("ActivityMinute == '"+data["Time"][i]+"'")['METs'])[0]
    print(hr,round(cal*4.184,2),mets)
    print("------------------------------------------------------------")

    if hr != data['HR(BPM)'][i] or cal != round(data['aEE(kJ/min)'][i]/4.184,2) or mets != data['METs'][i]:
      err_row.append((data["Time"][i],i))
  return err_row







def range_list(k):
  list_of = []
  main_list = []

  for i in range(len(k)-1):
 
    if k[i+1]-k[i] == 1:
      list_of.append(k[i])
    else:
      list_of.append(k[i])
      main_list.append(list_of)
      list_of=[]
    if i == len(k)-2:
      if k[i+1]-k[i] == 1:
        list_of.append(k[i+1])
        main_list.append(list_of)
      else:
        main_list.append([k[i+1]])
  
  return main_list






def mets_minute_refine(data,minutes):

  mets_10 = []
  mets_more = []
  for i in range(data.shape[0]):
    if data["METs"][i] > 10:
      mets_more.append(i)
    else:
      mets_10.append(i)

  main_l = range_list(mets_more)

  removable_list = []
  for i in range(len(main_l)-1):
    if len(main_l[i]) < minutes:
      for j in main_l[i]:
        removable_list.append(j)

  df1=data.drop(data.index[mets_10+removable_list]).reset_index()
  return df1





from datetime import datetime

def formate24(t):
  time = t.split(" ")
  m2 = time[1]+" "+time[2]
  in_time = datetime.strptime(m2, "%I:%M:%S %p")
  out_time = datetime.strftime(in_time, "%H:%M:%S")
  return out_time


def dif_min(start_time, end_time):
  t1 = datetime.strptime(start_time, "%H:%M:%S")
  t2 = datetime.strptime(end_time, "%H:%M:%S")

  delta = t2 - t1

  return delta.total_seconds()/60
     


In [5]:
subject = '001-JV'




for i in glob(path+"/*"):
  #get_contenct(i)
  if subject in i.split("/"):
    Post_1min,Post_cal,Post_met, Pre_1min,Pre_cal,Pre_met = get_content(i)

df_post_1min = pd.read_csv(Post_1min)
df_post_cal = pd.read_csv(Post_cal)
df_post_met = pd.read_csv(Post_met)

df_pre_1min = pd.read_csv(Pre_1min)
df_pre_cal = pd.read_csv(Pre_cal)
df_pre_met = pd.read_csv(Pre_met)



In [8]:
minute = 5

test_data = mets_minute_refine(df_post_met,minute)

#test_data.to_csv("Test_METs_Series_001-JV.csv",index = False)

In [9]:
test_data.head()

,index,ActivityMinute,METs
0,1014,12/17/2021 4:54:00 PM,11
1,1015,12/17/2021 4:55:00 PM,24
2,1016,12/17/2021 4:56:00 PM,15
3,1017,12/17/2021 4:57:00 PM,14
4,1018,12/17/2021 4:58:00 PM,14


In [19]:
test_data.columns

Index(['index', 'ActivityMinute', 'METs'], dtype='object')

In [10]:

kill = list(test_data['index'])

let = range_list(kill)

In [23]:
lol = [1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027]

In [35]:

time_diff = []
for i in tqdm(let):

  start_time = formate24(list(test_data.query("index == "+str(i[0])+"")['ActivityMinute'])[0])
  end_time = formate24(list(test_data.query("index == "+str(i[-1])+"")['ActivityMinute'])[0])
  diff = dif_min(start_time,end_time)
  if diff < 5 :
    time_diff.append([i[0],i[-1],diff])



In [36]:
print(*time_diff,sep="\n")

[1712, 1716, 4.0]
[1745, 1749, 4.0]
[1933, 1937, 4.0]
[1958, 1962, 4.0]
[2243, 2247, 4.0]
[2289, 2293, 4.0]
[2395, 2399, 4.0]
[2629, 2633, 4.0]
[3553, 3557, 4.0]
[3588, 3592, 4.0]
[3700, 3704, 4.0]
[4006, 4010, 4.0]
[4255, 4259, 4.0]
[4854, 4858, 4.0]
[5889, 5893, 4.0]
[6750, 6754, 4.0]
[6809, 6813, 4.0]
[7035, 7039, 4.0]
[7704, 7708, 4.0]
[9373, 9377, 4.0]
[9455, 9459, 4.0]
[10377, 10381, 4.0]
[10659, 10663, 4.0]
[10710, 10714, 4.0]
[10754, 10758, 4.0]
[11285, 11289, 4.0]
[11327, 11331, 4.0]
[11341, 11345, 4.0]
[12135, 12139, 4.0]
[12217, 12221, 4.0]
[12244, 12248, 4.0]
[12467, 12471, 4.0]
[12637, 12641, 4.0]
[12917, 12921, 4.0]
[15395, 15399, 4.0]
[15422, 15426, 4.0]
[16216, 16220, 4.0]
[16444, 16448, 4.0]
[16863, 16867, 4.0]
[18059, 18063, 4.0]
[18184, 18188, 4.0]
[18247, 18251, 4.0]
[19670, 19674, 4.0]
[19849, 19853, 4.0]
[19882, 19886, 4.0]
[20230, 20234, 4.0]
[21587, 21604, -1423.0]
[22669, 22673, 4.0]
[22977, 22981, 4.0]
[23573, 23577, 4.0]
[26906, 26910, 4.0]
[27135, 27139, 4.0